<br><br><br>
# <center>Functional Programming #16</center>




### <center>2018. 8. 27.</center>
<br><br><br>

In [1]:
val add1: Int => Int = _ + 1
val multiply3: Int => Int = _ * 3
// ---
val add: Int => Int => Int = x => y => x + y
val multiply: Int => Int => Int = x => y => x * y
// ---
def map[A, B]: (A => B) => List[A] => List[B] = f => {
    case Nil   => Nil
    case x::xs => f(x) :: map(f)(xs)
}
// ---
def concat[A]: List[List[A]] => List[A] = {
    case Nil   => Nil
    case x::xs => x ::: concat(xs)
}
// ---
sealed trait Tree[+A]
case class Node[A](l: Tree[A], a: A, r: Tree[A]) extends Tree[A]
case object Empty extends Tree[Nothing]

object Node { def apply[A](l: Tree[A], a: A, r: Tree[A]): Tree[A] = new Node(l, a, r) }
// ---
sealed trait Maybe[+A]
case class Just[A](a: A) extends Maybe[A]
case object Nahthing extends Maybe[Nothing]

object Just { def apply[A](a: A): Maybe[A] = new Just(a) }
// ---
def foldr[A, B]: (A => B => B) => B => List[A] => B = f => v => {
    case Nil   => v
    case x::xs => f(x)(foldr(f)(v)(xs))
}
// ---
trait Functor[F[+_]] {
    def fmap[A, B]: (A => B) => F[A] => F[B]
}

def fmap[A, B, F[+A]: Functor](f: A => B)(fa: F[A]) = implicitly[Functor[F]].fmap(f)(fa)

class Function1Op[A, B](f: A => B) {
    def <&>[F[+A]: Functor](fa: F[A]) = implicitly[Functor[F]].fmap(f)(fa)
}
implicit def toFunction1Op[A, B](f: A => B) = new Function1Op(f)
// ---
implicit val maybeFunctor = new Functor[Maybe] {
    def fmap[A, B] = f => {
        case Just(a)  => Just(f(a))
        case Nahthing => Nahthing
    }
}
// ---
trait Applicative[F[+_]] extends Functor[F] {
    def pure[A]: A => F[A]
    def app[A, B]: F[A => B] => F[A] => F[B]
    def fmap[A, B]: (A => B) => F[A] => F[B] = f => fa => app(pure(f))(fa)
}

def pure[A, F[+A]: Applicative](a: A) = implicitly[Applicative[F]].pure(a)
def app[A, B, F[+A]: Applicative](ff: F[A => B])(fa: F[A]) = implicitly[Applicative[F]].app(ff)(fa)

class ApplicativeOp[A, B, F[+_]](ff: F[A => B], e: Applicative[F]) {
    def <*>(fa: F[A]) = e.app(ff)(fa)
}
implicit def toApplicativeOp[A, B, F[+_]: Applicative](ff: F[A => B]) = new ApplicativeOp(ff, implicitly[Applicative[F]])
// ---
implicit val maybeApplicative = new Applicative[Maybe] {
    def pure[A] = Just[A]
    def app[A, B] = ff => fa => (ff, fa) match {
        case (Just(f), Just(a))  => Just(f(a))
        case _                   => Nahthing
    }
}
// ---
implicit val listApplicative = new Applicative[List] {
    def pure[A] = a => List(a)
    def app[A, B] = ff => fa => (ff, fa) match {
        case (f::fs, _::_) => fmap(f)(fa) ::: app(fs)(fa)
        case _             => Nil
    }
    override def fmap[A, B] = f => {
        case Nil   => Nil
        case x::xs => f(x) :: fmap(f)(xs)
    }
}
// ---
trait Monoid[M] {
    def mempty: M
    def mappend: M => M => M
    def mconcat: List[M] => M = foldr[M, M](mappend)(mempty)
}

def mempty[M: Monoid] = implicitly[Monoid[M]].mempty
def mappend[M: Monoid](m1: M)(m2: M) = implicitly[Monoid[M]].mappend(m1)(m2)
def mconcat[M: Monoid](l: List[M]) = implicitly[Monoid[M]].mconcat(l)

class MonoidOp[M](m1: M, e: Monoid[M]) {
    def <>(m2: M) = e.mappend(m1)(m2)
}
implicit def toMonoidOp[M: Monoid](m1: M) = new MonoidOp(m1, implicitly[Monoid[M]])
// ---
implicit def listMonoid[A] = new Monoid[List[A]] {
    def mempty = Nil
    def mappend = m1 => m2 => m1 ::: m2
}

add1: Int => Int = $sess.cmd0Wrapper$Helper$$Lambda$3148/412565384@7e2e1c6b
multiply3: Int => Int = $sess.cmd0Wrapper$Helper$$Lambda$3149/617544893@15b30c12
add: Int => Int => Int = $sess.cmd0Wrapper$Helper$$Lambda$3150/1686373328@22290ef1
multiply: Int => Int => Int = $sess.cmd0Wrapper$Helper$$Lambda$3151/1580090632@4cad497c
defined function map
defined function concat
defined trait Tree
defined class Node
defined object Empty
defined object Node
defined trait Maybe
defined class Just
defined object Nahthing
defined object Just
defined function foldr
defined trait Functor
defined function fmap
defined class Function1Op
defined function toFunction1Op
maybeFunctor: AnyRef with Functor[Maybe] = $sess.cmd0Wrapper$Helper$$anon$3@512cd00d
defined trait Applicative
defined function pure
defined function app
defined class ApplicativeOp
defined function toApplicativeOp
maybeApplicative: AnyRef with Applicative[Maybe] = $sess.cmd0Wrapper$Helper$$anon$1@71b6d4da
listApplicative: AnyRef with Appl

<br><br><br>
# <center>Monad</center>
<br><br><br>

In [2]:
trait Monad[M[+_]] extends Applicative[M] {
    def reeturn[A]: A => M[A]
    def bind[A, B]: M[A] => (A => M[B]) => M[B]
    def then[A, B]: M[A] => M[B] => M[B] = x => y => bind(x)(_ => y)
//     def fail[A]: String => M[A] = msg => error(msg)
    def pure[A] = reeturn[A]
    def withFilter[A]: M[A] => (A => Boolean) => M[A]
}

def reeturn[A, M[+A]: Monad](a: A) = implicitly[Monad[M]].reeturn(a)

class MonadOp[A, M[+_]](ma: M[A], e: Monad[M]) {
    def >>=[B](f: A => M[B]) = e.bind(ma)(f)
    def >>[B](mb: M[B]) = e.then(ma)(mb)
    def flatMap[B](f: A => M[B]) = e.bind(ma)(f)
    def map[B](f: A => B) = e.fmap(f)(ma)
    def withFilter(p: A => Boolean): M[A] = e.withFilter(ma)(p)
}
implicit def toMonadOp[A, M[+_]: Monad](ma: M[A]) = new MonadOp(ma, implicitly[Monad[M]])

defined trait Monad
defined function reeturn
defined class MonadOp
defined function toMonadOp

<br><br><br>
# <center>Maybe monad</center>
<br><br><br>

In [3]:
implicit val maybeMonad = new Monad[Maybe] {
    def app[A, B] = maybeApplicative.app

    def reeturn[A] = Just[A]
    def bind[A, B] = {
        case Just(a)  => f => f(a)
        case Nahthing => _ => Nahthing
    }

    def withFilter[A] = {
        case Just(a)  => p => if (p(a)) Just(a) else Nahthing
        case Nahthing => _ => Nahthing
    }
}

maybeMonad: AnyRef with Monad[Maybe] = $sess.cmd2Wrapper$Helper$$anon$1@40c6f427

In [4]:
val madd1: Int => Maybe[Int] = x => Just(x + 1)
val mmultiply3: Int => Maybe[Int] = x => Just(x * 3)

reeturn(9) >>= madd1 >>= mmultiply3 >>= madd1 >>= mmultiply3 >>= madd1
(((((reeturn(9) >>= madd1) >>= mmultiply3) >>= madd1) >>= mmultiply3) >>= madd1)

1 :: 2 :: 3 :: 4 :: 5 :: Nil
(1 :: (2 :: (3 :: (4 :: (5 :: Nil)))))

madd1: Int => Maybe[Int] = $sess.cmd3Wrapper$Helper$$Lambda$3426/13907036@75fbb3cf
mmultiply3: Int => Maybe[Int] = $sess.cmd3Wrapper$Helper$$Lambda$3427/1429009457@16bbe9f
res3_2: Maybe[Int] = Just(94)
res3_3: Maybe[Int] = Just(94)
res3_4: List[Int] = List(1, 2, 3, 4, 5)
res3_5: List[Int] = List(1, 2, 3, 4, 5)

In [5]:
val foo: Int => Maybe[Int] = reeturn(_) >>= madd1 >>= mmultiply3 >>= madd1 >>= mmultiply3 >>= madd1

foo: Int => Maybe[Int] = $sess.cmd4Wrapper$Helper$$Lambda$3467/1412207006@2a4c6ed7

In [6]:
val foo: Int => Maybe[Int] = reeturn(_) >>= madd1 >>= mmultiply3 >>= madd1 >>= mmultiply3 >>= madd1

val bar: Int => Maybe[Int] = x => reeturn(x) >>= madd1 >>= mmultiply3 >>= madd1 >>= mmultiply3 >>= madd1

foo: Int => Maybe[Int] = $sess.cmd5Wrapper$Helper$$Lambda$3473/1901594205@22b0af85
bar: Int => Maybe[Int] = $sess.cmd5Wrapper$Helper$$Lambda$3474/1784074021@55989c5

<br><br><br>
# <center>Cats</center>
<center>
![Cats](../img/cats2.png)<br>
[Cats examples](Cats.ipynb)
</center>
<br><br><br>

In [7]:
import $ivy.`org.typelevel::cats-core:1.1.0`

import $ivy.$                               

<br><br><br>
# <center>Type classes - Cats</center>
<br><br><br>

In [8]:
def sumInts(list: List[Int]): Int = list.foldRight(0)(_ + _)

def concatStrings(list: List[String]): String = list.foldRight("")(_ ++ _)

def unionSets[A](list: List[Set[A]]): Set[A] = list.foldRight(Set.empty[A])(_ union _)

defined function sumInts
defined function concatStrings
defined function unionSets

<br>
# <center>Type classes - Cats</center>
---
<br>
## <center>Instances</center>
<br><br><br>

In [8]:
import cats.Monoid

new Monoid[String] {}

cmd8.sc:3: object creation impossible, since:
it has 2 unimplemented members.
/** As seen from <$anon: cats.Monoid[String]>, the missing signatures are as follows.
 *  For convenience, these are usable as stub implementations.
 */
  // Members declared in cats.kernel.Monoid
  def empty: String = ???
  
  // Members declared in cats.kernel.Semigroup
  def combine(x: String,y: String): String = ???

val res8_1 = new Monoid[String] {}
                 ^

: 

In [9]:
import cats.Monoid

val intAdditionMonoid = new Monoid[Int] {
    def empty: Int = 0
    def combine(x: Int, y: Int): Int = x + y
}

val intMultiplicationMonoid = new Monoid[Int] {
    def empty: Int = 1
    def combine(x: Int, y: Int): Int = x * y
}

val stringMonoid = new Monoid[String] {
    def empty: String = ""
    def combine(x: String, y: String): String = x ++ y
}

import cats.Monoid


intAdditionMonoid: AnyRef with cats.package.Monoid[Int] = $sess.cmd8Wrapper$Helper$$anon$1@36a9cc50
intMultiplicationMonoid: AnyRef with cats.package.Monoid[Int] = $sess.cmd8Wrapper$Helper$$anon$2@6c83974e
stringMonoid: AnyRef with cats.package.Monoid[String] = $sess.cmd8Wrapper$Helper$$anon$3@12139d02

<br>
# <center>Type classes - Cats</center>
---
<br>
## <center>Interface</center>
<br><br><br>

In [10]:
def combineAll[A](list: List[A], A: Monoid[A]): A = list.foldRight(A.empty)(A.combine)

combineAll(List(1, 2, 3, 4, 5), intAdditionMonoid)
combineAll(List(1, 2, 3, 4, 5), intMultiplicationMonoid)
combineAll(List("foo", "bar"), stringMonoid)

defined function combineAll
res9_1: Int = 15
res9_2: Int = 120
res9_3: String = "foobar"

In [11]:
implicit val stringMonoid = new Monoid[String] {
    def empty: String = ""
    def combine(x: String, y: String): String = x ++ y
}
// object Monoid {
//     def apply[A: Monoid]: Monoid[A] = implicitly[Monoid[A]]
// }

// def combineAll[A](list: List[A], A: Monoid[A]): A = list.foldRight(A.empty)(A.combine)
def combineAll[A: Monoid](list: List[A]): A = list.foldRight(Monoid[A].empty)(Monoid[A].combine)

combineAll(List("foo", "bar"))

stringMonoid: AnyRef with Monoid[String] = $sess.cmd10Wrapper$Helper$$anon$1@22df2855
defined function combineAll
res10_2: String = "foobar"

<br>
# <center>Type classes - Cats</center>
---
<br>
## <center>Semigroups</center>
<br><br><br>

In [11]:
import cats.Semigroup

new Semigroup[Int] {}

cmd11.sc:3: object creation impossible, since method combine in trait Semigroup of type (x: Int, y: Int)Int is not defined
val res11_1 = new Semigroup[Int] {}
                  ^

: 

In [12]:
import cats.Semigroup

implicit val intAdditionsSemigroup = new Semigroup[Int] {
    def combine(x: Int, y: Int) = x + y
}

Semigroup[Int].combine(2, 3)

Semigroup[Int].combine(1, Semigroup[Int].combine(2, 3))
Semigroup[Int].combine(Semigroup[Int].combine(1, 2), 3)

import cats.Semigroup


intAdditionsSemigroup: AnyRef with cats.package.Semigroup[Int] = $sess.cmd11Wrapper$Helper$$anon$1@60a0355e
res11_2: Int = 5
res11_3: Int = 6
res11_4: Int = 6

In [13]:
import cats.syntax.semigroup._

2 |+| 3

import cats.syntax.semigroup._


res12_1: Int = 5

In [13]:
val map1 = Map("Hello" -> 0, "World" -> 1)
val map2 = Map("Hello" -> 2, "Cats"  -> 3)

Semigroup[Map[String, Int]].combine(map1, map2)

map1 |+| map2

cmd13.sc:4: could not find implicit value for parameter ev: cats.kernel.Semigroup[Map[String,Int]]
val res13_2 = Semigroup[Map[String, Int]].combine(map1, map2)
                       ^cmd13.sc:6: value |+| is not a member of scala.collection.immutable.Map[String,Int]
val res13_3 = map1 |+| map2
                   ^

: 

In [14]:
import cats.instances.map._

val map1 = Map("Hello" -> 0, "World" -> 1)
val map2 = Map("Hello" -> 2, "Cats"  -> 3)

Semigroup[Map[String, Int]].combine(map1, map2)

map1 |+| map2

import cats.instances.map._


map1: Map[String, Int] = Map("Hello" -> 0, "World" -> 1)
map2: Map[String, Int] = Map("Hello" -> 2, "Cats" -> 3)
res13_3: Map[String, Int] = Map("Hello" -> 2, "Cats" -> 3, "World" -> 1)
res13_4: Map[String, Int] = Map("Hello" -> 2, "Cats" -> 3, "World" -> 1)

In [15]:
import cats.Semigroup
import cats.instances.all._

import cats.Semigroup

import cats.instances.all._

In [16]:
Semigroup[List[Byte]]
Semigroup[Set[Int]]

res15_0: Semigroup[List[Byte]] = cats.kernel.instances.ListMonoid@54075444
res15_1: Semigroup[Set[Int]] = cats.kernel.instances.SetSemilattice@2c12a743

In [17]:
val a: Byte = 0
val b: Byte = 1

List(a, b) |+| List(a, b, a)
Set(1, 2) |+| Set(2, 3, 4)

a: Byte = 0
b: Byte = 1
res16_2: List[Byte] = List(0, 1, 0, 1, 0)
res16_3: Set[Int] = Set(1, 2, 3, 4)

In [18]:
trait Foo
Semigroup[List[Foo]]

defined trait Foo
res17_1: Semigroup[List[Foo]] = cats.kernel.instances.ListMonoid@40148cea

In [19]:
val foo = new Foo {}
val bar = new Foo {}

List(foo) |+| List(bar)

foo: AnyRef with Foo = $sess.cmd18Wrapper$Helper$$anon$1@7740591d
bar: AnyRef with Foo = $sess.cmd18Wrapper$Helper$$anon$2@1323614f
res18_2: List[AnyRef with Foo] = List(
  $sess.cmd18Wrapper$Helper$$anon$1@7740591d,
  $sess.cmd18Wrapper$Helper$$anon$2@1323614f
)

In [20]:
Semigroup[(List[Foo], Int)]

res19: Semigroup[(List[Foo], Int)] = cats.kernel.instances.TupleInstances2$$anon$182@3acc0dec

In [21]:
(List(foo), 1) |+| (List(foo, bar), 2)

res20: (List[AnyRef with Foo], Int) = (
  List(
    $sess.cmd18Wrapper$Helper$$anon$1@7740591d,
    $sess.cmd18Wrapper$Helper$$anon$1@7740591d,
    $sess.cmd18Wrapper$Helper$$anon$2@1323614f
  ),
  3
)